In [33]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import argrelmax, savgol_filter
from scipy.io import loadmat

import sys
sys.path.append('/home/jleland/Coding/Projects/flopter/')

import flopter.spice.splopter as spl
import flopter.core.fitters as f
import flopter.core.constants as c
import flopter.core.normalise as n
import flopter.spice.tdata as sd
import analysis.scripts.spice.spicerun as spr
import flopter.core.ivdata as ivd

In [6]:
# splopter = spl.Splopter('bin/data_c/', 'test/', 'cumslow_test1/')
# splopter = spl.Splopter('bin/data_m/', 'magnum/', 'fetail_T0_S/backup_20190404-1430/')
# splopter = spl.Splopter('bin/data_local_m/', 'magnum/', 'fetail_T0_S_mkommtest/')
splopter = spl.Splopter('bin/data_m/', 'lowdens_anglescan/', 'flush_trunc1/')

In [30]:
splopter = spl.Splopter('bin/data_local_m/', 'magnum/', 'fetail_T0_S_legacytest/')

No a-file given, continuing without


In [10]:
regions = {
    'Sheath': [281, 301, 0, 212],
    'Injection': [675, 685, 0, 2776],
    'Lower-mid': [381, 401, 0, 212],
    'Mid': [481, 501, 0, 212],
    'Upper-mid': [581, 601, 0, 212]
}

In [8]:
# splopter.prepare(homogenise=False, make_denormaliser=False)
splopter.prepare(homogenise=True, make_denormaliser=True)

Sheath edge temperature ratio is: 1.5685269192518887


/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [11]:
spr.run_multi_hist_analysis(regions=regions, splopter=splopter, show_fl=True, species=2)

/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/numpy/lib/histograms.py:893: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges


ValueError: Shape of array too small to calculate a numerical gradient, at least (edge_order + 1) elements are required.

In [35]:
print(len(splopter.tdata.diagnostics[c.DIAG_PROBE_POT]))
splopter.prepare(homogenise=True, make_denormaliser=False)

1
Cannot homogenise, data has already been homogenised!


In [30]:
# splopter.plot_iv(plot_e=True, plot_i=True, plot_tot=True, plot_vf=True)
plt.figure()
plt.plot(splopter.iv_data['V'][:-1], splopter.iv_data['I'][:-1])
plt.xlabel('')

plt.figure()
plt.plot(splopter.raw_data['V'][:-1])
plt.plot(splopter.raw_data['I'][:-1])


In [14]:
plt.figure()
plt.contourf(splopter.tdata.objectsenum)
plt.colorbar()

In [15]:
for raw_data_label, raw_data_arr in splopter.raw_data.items():
    print(raw_data_label, len(raw_data_arr))
    
plt.figure()
plt.plot(splopter.raw_data['V'])

I 1584
V 1584
t 1584
I_e 1584
I_i 1584
sigma 1584


In [37]:
plt.figure()
plt.plot(splopter.raw_data['I'], label=r'$I$')
plt.plot(splopter.raw_data['I_e'], label=r'$I_e$')
plt.plot(splopter.raw_data['I_i'], label=r'$I_i$')
plt.legend()

TypeError: 'NoneType' object is not subscriptable

In [21]:
splopter.plot_raw(plot_list=['I', 'I_e', 'I_i'], show_fl=True)

In [32]:
plt.figure()
plt.plot(splopter.tdata.diagnostics[c.DIAG_PROBE_POT])

In [38]:
V = splopter.tdata.diagnostics[c.DIAG_PROBE_POT]
I_e = splopter.tdata.objectscurrente[2] + splopter.tdata.objectscurrente[3]
I_i = splopter.tdata.objectscurrenti[2] + splopter.tdata.objectscurrenti[3]

print(len(I_e), len(I_i))
print(len(V))

1488 1488
1


In [39]:
plt.figure()
plt.plot(I_e + I_i)

In [32]:
splopter.plot_2d_variable()


In [21]:
splopter.iv_data

TypeError: 'IVData' object is not callable

In [23]:
str_sec = np.where(splopter.iv_data[c.POTENTIAL] <= -30)
i_ss = splopter.iv_data[c.CURRENT][str_sec]
splopter.iv_data[c.SIGMA] = np.std(i_ss) * np.ones_like(splopter.iv_data[c.CURRENT])

/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [28]:
# Run only once to get rid of outlier final values.
for data_label, data_arr in splopter.iv_data.items():
    splopter.iv_data[data_label] = data_arr[:-1]

In [32]:
fit_data = splopter.iv_data.multi_fit() # <-- this doesn't seem to work
fit_data.plot()

ValueError: Internal work array size computation failed: -5

In [46]:
new_iv_data = ivd.IVData(splopter.denormaliser(splopter.iv_data[c.POTENTIAL], c.CONV_POTENTIAL),
                         splopter.denormaliser(splopter.iv_data[c.CURRENT], c.CONV_CURRENT)[0],
                         splopter.denormaliser(splopter.iv_data[c.TIME], c.CONV_TIME),
                         e_current=splopter.denormaliser(splopter.iv_data[c.ELEC_CURRENT], c.CONV_CURRENT)[0],
                         i_current=splopter.denormaliser(splopter.iv_data[c.ION_CURRENT], c.CONV_CURRENT)[0],
                         sigma=splopter.denormaliser(splopter.iv_data[c.SIGMA], c.CONV_CURRENT)[0], estimate_error_fl=False)

In [47]:
for data_label, data_arr in new_iv_data.items():
    print(data_label, np.shape(data_arr))

I (526,)
V (526,)
t (1583,)
I_e (526,)
I_i (526,)
sigma (526,)


In [49]:
fig, ax = plt.subplots()
ax.plot(new_iv_data[c.POTENTIAL], new_iv_data[c.CURRENT])